In [19]:
import pandas as pd


In [20]:
df = pd.read_csv("ibd_source_data.csv")
df.head()

,AUTO_ID,PAT_ID,PAT_MRN_ID,MARITAL_STATUS,EMPLOYMENT_STATUS,GENDER,AGE,DISTANCE_KM,SIBDQ,ENC_CONSULT,...,ANTI_IL12,ANTI_INTEGRIN,ANTI_TNF,Immunomodulators,Systemic_steroids,Vitamin_D,ANTIBIOTICS,PSYCH_DX,TOBACCO_USE,CHARGE_FLAG_85pctile
0,0,156369,156369,Married,Employed,Male,Age3,Dist3,Good,Never,...,Never,Never,Never,Persistent,Persistent,Never,Never,NaN,Quit,0.0
1,1,174687646,841189757,Single,NotEmployed,Male,Age1,Dist1,Good,Never,...,Never,Never,Never,Never,Never,Never,Persistent,NaN,Yes,0.0
2,2,738090216,738090216,Married,NotEmployed,Female,Age1,Dist3,Good,Never,...,Never,Never,Never,Persistent,Never,Never,Transient,NaN,Never,0.0
3,3,176623065,841748158,Single,Employed,Female,Age2,Dist1,NaN,Never,...,Never,Never,Never,Never,Never,Transient,Transient,NaN,Never,0.0
4,4,60187,60187,Married,Employed,Female,Age3,Dist2,Bad,Never,...,Never,Never,Never,Never,Never,Never,Never,NaN,Never,0.0


In [24]:
df.shape

(3143, 32)

In [33]:
result = df.query("MARITAL_STATUS=='Married'")['GENDER'].value_counts()

In [36]:
result.sum()

dtype('int64')

In [28]:
df['GENDER'].value_counts().sum()

3143

In [21]:
df.columns

Index(['AUTO_ID', 'PAT_ID', 'PAT_MRN_ID', 'MARITAL_STATUS',
       'EMPLOYMENT_STATUS', 'GENDER', 'AGE', 'DISTANCE_KM', 'SIBDQ',
       'ENC_CONSULT', 'ENC_OFF', 'ENC_PROC', 'ENC_TEL', 'EOS_EVER_HIGH',
       'MONO_EVER_HIGH', 'ALBUMIN', 'HEMO', 'ESR', 'CRP', 'VITD_LAB', 'PSYCH',
       '5_ASA', 'ANTI_IL12', 'ANTI_INTEGRIN', 'ANTI_TNF', 'Immunomodulators',
       'Systemic_steroids', 'Vitamin_D', 'ANTIBIOTICS', 'PSYCH_DX',
       'TOBACCO_USE', 'CHARGE_FLAG_85pctile'],
      dtype='object')

In [22]:
def get_split_data(data, col_list):
    #sel = data.groupby(col_list, as_index=False).count()
    sel_col_list = col_list
    sel_col_list.append('AUTO_ID')
    sel = data[sel_col_list]
    sel = sel.groupby(col_list).count()

    
    return sel

In [23]:
sel_data_cols = ['GENDER','MARITAL_STATUS', 'AUTO_ID']
sel_data = df[sel_data_cols]
#df.groupby(['GENDER','MARITAL_STATUS']).count()
sel_data.groupby(['GENDER','MARITAL_STATUS']).count().to_json()


'{"AUTO_ID":{"["Female","Married"]":919,"["Female","Single"]":718,"["Female","Unknown"]":31,"["Male","Married"]":765,"["Male","Single"]":662,"["Male","Unknown"]":41}}'

In [6]:
col = 'EMPLOYMENT_STATUS'
q = "GENDER=='Male' & MARITAL_STATUS == 'Married'"
df.query(q)[col].value_counts().to_json()


'{"Employed":477,"NotEmployed":163,"Unknown":116,"Student":9}'

In [18]:
is_root = False
col = 'EMPLOYMENT_STATUS'
q = "GENDER=='Male' & MARITAL_STATUS == 'Married'"
#df.query(q)[col].value_counts().to_json()


children = []
result = df.query(q)[col].value_counts()
for key, val in result.items():
    temp = {'name' : key, 'weight' : val, 'children' : []}
    children.append(temp)
    
if is_root:
    container = {'name' : col, 'children' : children}
    
else:
    container = children
    
print(container)
    

[{'name': 'Employed', 'weight': 477, 'children': []}, {'name': 'NotEmployed', 'weight': 163, 'children': []}, {'name': 'Unknown', 'weight': 116, 'children': []}, {'name': 'Student', 'weight': 9, 'children': []}]


In [87]:
# Required Python Machine learning Packages
import pandas as pd
import numpy as np

# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split

# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB

# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline

In [88]:
def label_encode(df, columns):
    for col in columns:
        le = LabelEncoder()
        col_values_unique = list(df[col].unique())
        le_fitted = le.fit(col_values_unique)
 
        col_values = list(df[col].values)
        #le.classes_
        col_values_transformed = le.transform(col_values)
        df[col] = col_values_transformed
        

In [93]:
features = df[['GENDER','MARITAL_STATUS','EMPLOYMENT_STATUS', 'AGE']]

label_encode(features, features.columns.values)

target = df[['Immunomodulators']]
label_encode(target, target.columns.values)



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [94]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.33, random_state = 10)


In [95]:
nb = GaussianNB()
nb.fit(features_train, target_train)
target_pred = nb.predict(features_test)
target_pred

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 0, ..., 0, 1, 1])

In [96]:
from sklearn import metrics
print(metrics.accuracy_score(target_test, target_pred))

0.4653179190751445
